In [4]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt


In [5]:
# Load dataset
file_path = "/kaggle/input/aggregated/final_merged_aggregated_data.csv"
df = pd.read_csv(file_path, parse_dates=["Date"], index_col="Date")


In [6]:
# Train-test split
TRAIN_SPLIT = int(len(df) * 0.90)  # 90% training data
train_dates = df.index[:TRAIN_SPLIT]
test_dates = df.index[TRAIN_SPLIT:]

In [7]:
# Scale the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(df.iloc[:TRAIN_SPLIT])  # Fit only on training data
df_scaled = pd.DataFrame(scaler.transform(df), columns=df.columns)
data = df_scaled.values  # Convert to numpy array

# Define hyperparameters
PAST_DAYS = 30   # Number of past days used for input
FUTURE_DAYS = 7  # Number of future days to predict


In [8]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data, past_days, future_days):
        self.data = data
        self.past_days = past_days
        self.future_days = future_days
        
        self.X, self.Y = self.create_sequences()

    def create_sequences(self):
        X, Y = [], []
        for i in range(len(self.data) - self.past_days - self.future_days):
            X.append(self.data[i:i + self.past_days])  # Past 30 days as input
            Y.append(self.data[i + self.past_days: i + self.past_days + self.future_days, 0])  # Next 9 days (target)
        
        return np.array(X), np.array(Y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return torch.tensor(self.X[index], dtype=torch.float32), torch.tensor(self.Y[index], dtype=torch.float32)

In [9]:
    
# Create training and test dataset
train_dataset = TimeSeriesDataset(data[:TRAIN_SPLIT], PAST_DAYS, FUTURE_DAYS)
test_dataset = TimeSeriesDataset(data[TRAIN_SPLIT:], PAST_DAYS, FUTURE_DAYS)

# Data Loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [10]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, output_size=FUTURE_DAYS):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, output_size)  # Output for the next 7 days

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))  # LSTM forward pass
        out = self.fc(out[:, -1, :])  # Get the last time-step's output
        return out


In [11]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize model, loss function, and optimizer
model = LSTMModel(input_size=data.shape[1]).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 60
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for X_batch, Y_batch in train_loader:
        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)

        optimizer.zero_grad()
        predictions = model(X_batch)
        loss = criterion(predictions, Y_batch)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")


Epoch [1/60], Loss: 0.0558
Epoch [2/60], Loss: 0.0410
Epoch [3/60], Loss: 0.0421
Epoch [4/60], Loss: 0.0409
Epoch [5/60], Loss: 0.0399
Epoch [6/60], Loss: 0.0395
Epoch [7/60], Loss: 0.0393
Epoch [8/60], Loss: 0.0390
Epoch [9/60], Loss: 0.0387
Epoch [10/60], Loss: 0.0384
Epoch [11/60], Loss: 0.0382
Epoch [12/60], Loss: 0.0375
Epoch [13/60], Loss: 0.0356
Epoch [14/60], Loss: 0.0303
Epoch [15/60], Loss: 0.0282
Epoch [16/60], Loss: 0.0257
Epoch [17/60], Loss: 0.0187
Epoch [18/60], Loss: 0.0153
Epoch [19/60], Loss: 0.0146
Epoch [20/60], Loss: 0.0142
Epoch [21/60], Loss: 0.0133
Epoch [22/60], Loss: 0.0120
Epoch [23/60], Loss: 0.0112
Epoch [24/60], Loss: 0.0107
Epoch [25/60], Loss: 0.0105
Epoch [26/60], Loss: 0.0103
Epoch [27/60], Loss: 0.0102
Epoch [28/60], Loss: 0.0102
Epoch [29/60], Loss: 0.0102
Epoch [30/60], Loss: 0.0102
Epoch [31/60], Loss: 0.0103
Epoch [32/60], Loss: 0.0104
Epoch [33/60], Loss: 0.0102
Epoch [34/60], Loss: 0.0101
Epoch [35/60], Loss: 0.0099
Epoch [36/60], Loss: 0.0097
E

In [12]:
model.eval()
predictions = []
actuals = []

with torch.no_grad():
    for X_batch, Y_batch in test_loader:
        X_batch = X_batch.to(device)
        Y_pred = model(X_batch).cpu().numpy().flatten()
        Y_actual = Y_batch.cpu().numpy().flatten()

        predictions.append(Y_pred)
        actuals.append(Y_actual)

# Convert to numpy arrays
predictions = np.array(predictions)
actuals = np.array(actuals)


In [15]:
import plotly.graph_objects as go

# Extract the last 7 test dates
test_dates_subset = test_dates[-8:-1]

# Ensure Y_actual and Y_pred are aligned
Y_actual_excluding_last = actuals[-1:]  
Y_pred_excluding_last = predictions[-1:]
fig = go.Figure()

# Actual values
fig.add_trace(go.Scatter(
    x=test_dates_subset, y=Y_actual_excluding_last.flatten(),
    mode='lines+markers',
    name='Actual Sales',
    line=dict(color='red'),
    marker=dict(symbol='x')
))

# Predicted values
fig.add_trace(go.Scatter(
    x=test_dates_subset, y=Y_pred_excluding_last.flatten(),
    mode='lines+markers',
    name='Predicted Sales',
    line=dict(color='blue')
))

fig.update_layout(
    title='Actual vs Predicted Sales',
    xaxis_title='Date',
    yaxis_title='Sales Value',
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    legend_title="Legend"
)

fig.show()


In [14]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Compute metrics
mae = mean_absolute_error(actuals, predictions)
mse = mean_squared_error(actuals, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(actuals, predictions)

# Print results
print("Evaluation Metrics of Model that includes weather features")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R² Score: {r2}")


Evaluation Metrics of Model that includes weather features
Mean Absolute Error (MAE): 0.06868802756071091
Mean Squared Error (MSE): 0.0068874238058924675
Root Mean Squared Error (RMSE): 0.08299050480127335
R² Score: 0.6215353252485997
